In [18]:
!pip install gradio

Defaulting to user installation because normal site-packages is not writeable


In [2]:
!pip install rouge

Defaulting to user installation because normal site-packages is not writeable


In [19]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


In [20]:
!pip install textstat

Defaulting to user installation because normal site-packages is not writeable


In [21]:
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable


In [22]:
!pip install TextBlob

Defaulting to user installation because normal site-packages is not writeable


In [23]:
from textblob import TextBlob

In [24]:
import requests
import json
import gradio
import os
from rouge import Rouge
from gensim.models import LsiModel
from gensim.similarities import MatrixSimilarity
from gensim import corpora
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from textstat import flesch_kincaid_grade
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [25]:
def calculate_compression_ratio(article, summary):
    original_length = len(article)
    summary_length = len(summary)
    compression_ratio = summary_length / original_length
    return compression_ratio

def calculate_rouge_score(reference, summary):
    rouge = Rouge()
    scores = rouge.get_scores(summary, reference)
    rouge_scores = scores[0]['rouge-1']['f']
    return rouge_scores

def calculate_lsa_similarity(input_text, summary):
    documents = [input_text, summary]
    texts = [[word for word in document.lower().split()] for document in documents]

    dictionary = corpora.Dictionary(texts)
    corpus = [dictionary.doc2bow(text) for text in texts]

    lsi_model = LsiModel(corpus, id2word=dictionary, num_topics=2)
    index = MatrixSimilarity(lsi_model[corpus])

    lsa_similarity = index[lsi_model[corpus[0]]][1]

    return lsa_similarity

def calculate_cosine_similarity(input_text, summary):
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform([input_text, summary])
    cosine_similarity_score = cosine_similarity(vectors[0], vectors[1])[0][0]
    return cosine_similarity_score

def calculate_flesch_kincaid(input_text, summary):
    flesch_kincaid_score = flesch_kincaid_grade(summary)
    return flesch_kincaid_score

# def calculate_unigram_similarity(input_text, summary, ngram_range=(1, 1)):
#     vectorizer = CountVectorizer(ngram_range= ngram_range)
#     vectors = vectorizer.fit_transform([input_text, summary])
#     ngram_similarity_score = cosine_similarity(vectors[0], vectors[1])[0][0]
#     return unigram_similarity_score
#     unigram_similarity_score = calculate_ngram_similarity(Input, Summary, ngram_range=(1, 1))
#     bigram_similarity_score = calculate_ngram_similarity(Input, Summary, ngram_range=(2, 2))

# def sentiment_analysis(text):
#     sentiment = TextBlob(text).sentiment
#     return sentiment.polarity, sentiment.subjectivity
#     sentiment_polarity, sentiment_subjectivity = sentiment_analysis(Summary)
#     return sentiment_polarity, sentiment_subjectivity

In [26]:
summary_data = pd.read_excel("summaries_v2.xlsx")
# input_texts = summary_data['Unnamed: 0'].values
input_texts = summary_data['article'].values


In [27]:
summary_bert = summary_data['bert'].values

In [28]:
# Read the Excel file
df = pd.read_excel('summaries_v2.xlsx')


# Create a list to store article and summary values in parallel
prompt = []

# Iterate over each row and store the values in parallel
for index, row in df.iterrows():
    prompt.append(('article is: ' + str(row['article']), '\nsummary is: ' + str(row['bert'])))

# Print the stored values
for value in prompt:
    print(value[0], value[1])

article is: (CNN) -- This Sunday, my family will be among the 110 million people gathered in front of their TVs to watch the Super Bowl, and with ad revenue projected at $300 million, we can expect advertisers to be as outrageous as ever, with spots promoting junk food, alcohol and erectile dysfunction drugs and depicting violence, sex and gender stereotypes. This said, TV ads -- even during the Super Bowl -- might be the least of parents' concerns. Digital media has dramatically changed the advertising landscape, and includes immersive websites, "advergaming," viral marketing, mobile ads, social media marketing and precise behavioral and location targeting. More than ever before, advertising and entertainment are inextricably linked. In many cases, the content is the ad, and this can be particularly confusing for children, who are less able to understand persuasive intent. Common Sense Media just released a research brief -- "Advertising to Children and Teens: Current Practices" -- to

In [29]:
model_list = list(summary_data.columns[1:])
scores = ['compression_ratio', 'rouge_score', 'lsa_similarity', 'cosine_similarity', 'flesch_kincaid']
scores_dict = {}
for i, score in enumerate(scores):
    scores_dict[i] = eval(f'calculate_{score}')
    
score_df = pd.DataFrame(columns = ['model'] + scores)

In [34]:
for model in model_list:
    summary_model = summary_data[model].values
    mean_score_row = []
    for score_key in list(scores_dict.keys()):
        score_individual = []
        for input_text, summary in zip(input_texts, summary_model):
            score_individual.append(scores_dict[score_key](input_text, summary))
        mean_score_row.append(sum(score_individual)/len(score_individual) )
    score_df.loc[len(score_df)] = [model] + mean_score_row


ValueError: cannot set a row with mismatched columns

In [31]:
score_df.loc[len(score_df)] = ['distilbart'] + mean_score_row


ValueError: cannot set a row with mismatched columns

In [30]:
score_df

,model,compression_ratio,rouge_score,lsa_similarity,cosine_similarity,flesch_kincaid
